In [1]:
import os
import shutil
import time
import pandas as pd
from tqdm.notebook import tqdm
import re
import file_function as ff
from os.path import join

### 파일명 작성 지침 변경에 따른 부산 수정

##### 파일 전체 옮기기

In [3]:
path = r'D:\부산 정리_v1'
dst_root = r'D:\부산 정리_v2'
ff.copy_all(path, dst_root)

20119538_임신호_초본.pdf
20407822_김대식_초본.pdf
20407852_정두남_원인서류.pdf
20407853_지종기_원인서류.pdf
20407854_장성_원인서류.pdf
20407855_변동희_원인서류.pdf
20407856_권성구_원인서류.pdf
20407858_권은진_원인서류.PDF
20407859_박애란_원인서류.pdf
20407859_박애란_원인서류_(2).PDF
20407859_박애란_채권양도통지서_1차.pdf
20407859_박애란_채권양도통지서_2차.pdf
20407859_박애란_초본.pdf
20407859_박애란_초본_(1).pdf
20407859_박애란_초본_(3).pdf
20407859_박애란_타채.pdf
20407859_박애란_판결문.pdf
20407859_박애란_판결문_재도.pdf
20407860_김세영_원인서류.pdf
20407860_김세영_원인서류_(2).pdf
20407860_김세영_지급명령.pdf
20407860_김세영_채권양도통지서_1차.pdf
20407860_김세영_채권양도통지서_2차.pdf
20407860_김세영_초본.pdf
20407861_문선효_원인서류.pdf
20407861_문선효_원인서류_(2).pdf
20407861_문선효_지급명령.pdf
20407861_문선효_채권양도통지서_1차.pdf
20407861_문선효_채권양도통지서_2차.pdf
20407861_문선효_초본.pdf
20407861_문선효_초본_(1).pdf
20407861_문선효_초본_(2).pdf
20407861_문선효_초본_(4).pdf
20407862_양홍주_원인서류.pdf
20407862_양홍주_원인서류_(2).pdf
20407862_양홍주_채권양도통지서_1차.pdf
20407862_양홍주_채권양도통지서_2차.pdf
20407862_양홍주_초본.pdf
20407863_김세경_등본.pdf
20407863_김세경_등본_(2).pdf
20407863_김세경_원인서류.pdf
20407863_김세경_원인서류_(2).pdf
20407863_김세경

##### '보증인', 집행권원, 강제집행 수정

In [2]:
path = r'D:\부산 정리_v2'
filelist = os.walk(path)
for root, dirs, files in filelist :
    for f in files :
        new_name = f

        if re.search("판결문|지급명령|이행권고|화해권고", new_name) :
            new_name = re.sub("판결문|지급명령|이행권고|화해권고", "집행권원", new_name)

        elif re.search("타채|결정문", new_name) :
            new_name = re.sub("타채|결정문", "강제집행", new_name)

        elif re.search("채권양도통지서", new_name) :
            new_name = re.sub("채권양도통지서", "양도통지서", new_name)

        if re.search("[_]+보증인", new_name) :
            split_list = re.sub("[_]+보증인", "", new_name).split("_")
            split_list[1] = "보증인" + " "+ split_list[1]

            new_name = "_".join(split_list)
        
        if new_name != f : 
            ff.re_name(os.path.join(root,f), os.path.join(r"C:\Users\SL\Desktop\result", new_name))
    

AttributeError: module 'file_function' has no attribute 're_name'

##### 사건번호 다시 넣기

### 체크하기

##### >>> 채무자키만

In [38]:
path = r'D:\부산 정리'
os.chdir(path)
file_list = os.walk(path)

p_key = re.compile(r'[\d]{8}[\D]')
p_notkey = re.compile('[0-9]{9,}')
error = []

for r, d, files in tqdm(file_list) :
    for f in files :
        if f != "Thumbs.db":
            if p_key.match(f) == None :
                error.append(f)
print(error)

0it [00:00, ?it/s]

[]


##### >>>문서종류만

In [64]:
p_docu = re.compile('원인서류|채권양도통지서|판결문|지급명령|이행권고|화해권고|타채|결정문|등본|초본|개회|신복|파산|외국인증명')

path = r'D:\부산 정리'
file_list = os.walk(path)
error_short = []
error_docu = []

for r, d, files in tqdm(file_list) :
    for f in files :
        if f != "Thumbs.db":
            splitname = f.split("_")
            if len(splitname)<3 :
                error_short.append(f)
            elif p_docu.search(splitname[2]) == None :
                error_docu.append(f)
print("short :", *error_short, sep="\n")
print("docu :", *error_docu, sep="\n")

0it [00:00, ?it/s]

short :
20459630ZONG WEI_외국인증명.pdf
docu :


### 돌발상황 처리

##### 이름 중간에 들어간 _

In [59]:
### 등,초본이 초본을 포함해버린다. 이거 수정해야
path = r'D:\부산 정리'

p_docu = re.compile(
    '원인서류|채권양도통지서|판결문|지급명령|이행권고|화해권고|타채|결정문|등본|초본|개회|신복|파산|외국인증명')
p_ = re.compile("_")

os.chdir(path)
file_list = os.walk(path)
fixed = []
count = 0

for r, d, files in tqdm(file_list):
    for f in files:
        if f != "Thumbs.db":
            new_name = ""
            if p_docu.search(f):
                st = p_docu.search(f).start() - 1
                new_name = f[:9] + p_.sub(" ", f[9:st]) + f[st:]  # 앞뒤 언더바 말고 중간에 언더바

                if f != new_name:
                    os.rename(r+"/"+f, r+"/"+new_name)
                    count += 1
                    fixed.append([f, new_name])
                    

print(count, "개의 파일 이름 변경 완료")
print(*fixed, sep="\n")

0it [00:00, ?it/s]

0 개의 파일 이름 변경 완료



##### 잘린 외국인 이름 정정

In [86]:
# 파일명에 보증인 키워드가 있는지만 보고 그거에 맞춰서 ... 

# 채무자조회파일 dict로 불러오기
dict = ff.debtorInfoDict()

path = r'D:\부산 정리'
file_list = os.walk(path)

p_eng = re.compile("[a-zA-Z]+")
p_etc = re.compile("보증인")

count = 0

for root, dir, files in tqdm(file_list) : 
    for f in files : 

        if f != "Thumbs.db":
            new_name = ""
            
            spl_name = f.split("_", maxsplit=2)

            key = spl_name[0]
            debtor_name = spl_name[1]
            others = spl_name[2]

            if p_eng.search(debtor_name) : # 이름에 영어 있는 경우
                if p_etc.search(others) : # 보증인 키워드 있는 경우
                    new_name = key +"_" + str(dict[key].보증인성명)+"_"+ others
                else : # 보증인 키워드 없는 경우
                    new_name = key +"_" + str(dict[key].성명)+"_"+others
                    
                if new_name != f :
                    src = os.path.join(root, f)
                    dst = os.path.join(root, new_name)
                    ff.re_name(src, dst)
                    count += 1
print(count, "개의 파일명 변경 완료")
                  

0it [00:00, ?it/s]

20412275_LE THANH HAI_외국인증명.pdf 20412275_레탕하이_외국인증명.pdf
20432140_YANGXUEHUI_원인서류.pdf 20432140_YANG XUEHUI_원인서류.pdf
20432235_PIXIANGXU_원인서류.pdf 20432235_PI XIANGXU_원인서류.pdf
20432249_THOMASWHALLEY_원인서류.pdf 20432249_THOMAS WHALLEY_원인서류.pdf
20432299_LICHUNHUA_원인서류.pdf 20432299_LI CHUNHUA_원인서류.pdf
20432493_YIJAERAN(이제란)_판결문.pdf 20432493_YI JAE RAN_판결문.pdf
20432493_YIJAERAN_원인서류.pdf 20432493_YI JAE RAN_원인서류.pdf
20432503_SPOHRRYANAUSTIN(조성수)_판결문.pdf 20432503_SPOHR RYAN AUSTIN_판결문.pdf
20432503_SPOHRRYANAUSTIN_원인서류.pdf 20432503_SPOHR RYAN AUSTIN_원인서류.pdf
20432517_XUJI_원인서류.pdf 20432517_XU JI_원인서류.pdf
20432533_KIMANTHONYJONGUK_원인서류.pdf 20432533_KIM ANTHONY JONGUK_원인서류.pdf
20432543_LIANGRENLONG_원인서류.pdf 20432543_LIANG RENLONG_원인서류.pdf
20432551_SAMCHANTHOL_원인서류.pdf 20432551_SAM CHANTHOL_원인서류.pdf
20432560_PIAOXIUYU_원인서류.pdf 20432560_PIAO XIUYU_원인서류.pdf
20432623_LINSHENGLONG_원인서류.pdf 20432623_LIN SHENGLONG_원인서류.pdf
20432624_KIMNATALIA_원인서류.pdf 20432624_KIM NATALIA_원인서류.pdf
20432677_PAIKSEUNGHO_원인서류.